# **Code for inference**

This notebook focuses on two main tasks:

1. **Inference for mT5 on English → Latin Translation**: Using a trained mT5 model to generate Latin translations from English inputs.
2. **Inference on Summaries with Mistral**: Generating extractive summaries using the Mistral-7B-Instruct model.

---

## **Table of Contents**

### 1. Inference for mT5 on Translation ('en' → 'la')
- 1.1 **Load Libraries**
- 1.2 **Define Global Parameters**
- 1.3 **Load the Trained Tokenizer and Model**
- 1.4 **Generate Translations**

### 2. Inference on Summaries with Mistral ('en' → 'en')
- 2.1 **Load Mistral Model for Summarization**
- 2.2 **Generate Summaries with Mistral**

---

## Inference for mT5 on translation 'en' -> 'la' along with the extraction of summaries

#### Librairies

In [26]:
import os
import pandas as pd
import numpy as np
import torch
import warnings
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
from utils.mT5_train import training_loop, plot_training
from utils.generate_translation import generate_translation, generate_translation_with_options, inference_from_csv, inference_from_csv_adding_column
from peft import LoraConfig, get_peft_model, PeftModel
from utils.bleu import calculate_bleu_and_chrf

#### Verify train set

In [27]:
# path_to_train = "/Data/AxelDlv/LatinSummarizer/prompt_no_stanza_train.csv"
# path_to_test = "/Data/AxelDlv/LatinSummarizer/prompt_no_stanza_test.csv"
# path_to_special_tokens = "/Data/AxelDlv/LatinSummarizer/common_tags_la_en.csv"  

path_to_train = "/Data/AxelDlv/LatinSummarizer/prompt_with_stanza_train.csv"
path_to_test = "/Data/AxelDlv/LatinSummarizer/prompt_with_stanza_test.csv"
path_to_special_tokens = "/Data/AxelDlv/LatinSummarizer/stanza_merged_tags_la_en.csv"

# Load data
train_data = pd.read_csv(path_to_train)
test_data = pd.read_csv(path_to_test)

In [28]:
# print samples of coupleq prompt / answer 
for i in range(5):
    print(train_data.iloc[i]['prompt'])
    print(train_data.iloc[i]['answer'])
    print()

train_data

<la> <no_stanza> rogabis eum et exaudiet te et vota tua reddes <la> <la.en> <en> <no_stanza> 
Thou shalt pray to him, and he will hear thee, and thou shalt pay vows. <en>

<en> <no_stanza> Here I ask, if sufficient protection is afforded to Hiempsal by the treaty  and if the Recentoric district is private property, what was use of excepting these lands by  name in the law? If that treaty itself has some obscurity in it, and if the Recentoric is  sometimes said to be public property, who do you suppose will believe that there have been two  interests found in the world, and only two, which he spared for nothing? Does there appear to  have been any coin in the world so carefully hidden that the architects of this law have failed  to scent it out? They are draining the provinces, the free cities, our allies, our friends, and  even the kings who are confederate with us. They are laying bands on the revenue of the Roman  people. <en> <en.la> <la> <no_stanza> 
hic quaero, si Hiempsali satis 

,prompt,answer,prefix
0,<la> <no_stanza> rogabis eum et exaudiet te et...,"Thou shalt pray to him, and he will hear thee,...",la.en
1,"<en> <no_stanza> Here I ask, if sufficient pro...","hic quaero, si Hiempsali satis est cautum foe...",en.la
2,<en> <no_stanza> Africans and warlike Spaniard...,"ast hic, tranquillo qua labitur agmine flumen,...",en.la
3,<en> <no_stanza> For who does not realize that...,"Quis est enim qui hoc non intellegat, nisi Cae...",en.la
4,"<la> 9 Sed dicebat, quod aliquis dicitur vider...","9 Sed dicebat, quod aliquis dicitur videre non...",la.la
...,...,...,...
633514,<la> <with_stanza> Duo <NUM> tamen <ADV> agger...,"However <ADV> , <PUNCT> two <NUM> lofty <ADJ> ...",la.en
633515,"<la> O homo, audi et intellige verba illius qu...","O homo, audi et intellige verba illius qui era...",la.la
633516,<en> <with_stanza> And <CCONJ> he <PRON> shall...,et reget illas in virga ferrea tamquam vas fig...,en.la
633517,"<la> ad 7 Ad septimum dicendum, quod duplicite...","ad 7 Ad septimum dicendum, quod dupliciter ali...",la.la


In [29]:
# print samples of couples prompt / answer
for i in range(5):
    print(test_data.iloc[i]['prompt'])
    print(test_data.iloc[i]['answer'])
    print()

test_data

<la> <no_stanza> Aperuerat iam Italiam bellumque transmiserat, ut supra memoravimus, ala Siliana, nullo apud quemquam Othonis favore, nec quia Vitellium mallent, sed longa pax ad omne servitium fregerat facilis occupantibus et melioribus incuriosos. <la> <la.en> <en> <no_stanza> 
The road into Italy had already been opened and the war transferred there by Siliuss cavalry, as we have said above. Although no one favoured Otho there, this success was not due to the preference of the people for Vitellius; but long peace had broken their spirits, so that they were ready for any kind of servitude, an easy prey to the first comer and careless as to who had the better cause. <en>

<la> <no_stanza> Usus practicus unus est ad altitudinem geopotentialis in meteorologia. <la> <la.en> <en> <no_stanza> 
One practical use is for altitude of geopotential heights in meteorology. <en>

<la> <with_stanza> praeferunt <VERB> gustandi <VERB> discretionem <NOUN> : <PUNCT> tamquam <SCONJ> non <PART> plurimum 

,prompt,answer,prefix
0,<la> <no_stanza> Aperuerat iam Italiam bellumq...,The road into Italy had already been opened an...,la.en
1,<la> <no_stanza> Usus practicus unus est ad al...,One practical use is for altitude of geopotent...,la.en
2,<la> <with_stanza> praeferunt <VERB> gustandi ...,As if we were not far inferior in this even to...,la.en
3,<en> <no_stanza> But let all things be done de...,omnia autem honeste et secundum ordinem fiant ...,en.la
4,"<la> Affert secundus ramus persicum, ex persic...","Affert secundus ramus persicum, ex persico, et...",la.la
...,...,...,...
33338,"<la> <no_stanza> Sed vos religiosi, qui eam qu...",But it is you who are the really religious peo...,la.en
33339,<la> <no_stanza> ne forte decepti faciatis vob...,"Lest ye corrupt yourselves, and make you a gra...",la.en
33340,<en> <with_stanza> And <CCONJ> thou <PRON> sha...,et diliges Dominum Deum tuum ex toto corde tuo...,en.la
33341,"<la> Hanc fabulam ponit Lucanus: Fuit, inquit,...","Hanc fabulam ponit Lucanus: Fuit, inquit, in L...",la.la


In [30]:
pd.read_csv(path_to_special_tokens)

,token
0,<en>
1,<la>
2,<en.la>
3,<la.en>
4,<la.la>
5,<with_stanza>
6,<no_stanza>
7,<clue>
8,<EMPTY>
9,<PAD>


#### Global parameters

In [ ]:
# mT5 model and tokenizer
max_seq_len = 412                                   # Maximum number of tokens of the input sequence
max_new_tokens = 412                                # Maximum number of tokens of the generated sequence
model_name = "/Data/AxelDlv/mt5-small-en-la-translation/mt5-small"
# checkpoint_path = f"/Data/AxelDlv/mt5-small-en-la-translation/mt5-small-en-la-translation-final_no_stanza-last_epoch"
checkpoint_path = f"/Data/AxelDlv/mt5-small-en-la-translation/mt5-small-en-la-translation-final_with_stanza-last_epoch"
STANZA = False

warnings.filterwarnings("ignore")

#### Inference

In [32]:
# Load special tokens and initialize tokenizer
special_tokens = pd.read_csv(path_to_special_tokens)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens['token'].tolist()})

# Load base model (without LoRA yet)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()
base_model.resize_token_embeddings(len(tokenizer))

# Define LoRA Configuration (MUST MATCH training config)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],  # LoRA layers added to 'q' and 'v' attention components
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Wrap base model with LoRA
model = PeftModel(base_model, lora_config)
model.load_state_dict(torch.load(f"{checkpoint_path}.pt", map_location="cuda"))
model.to("cuda")

# Set model to eval mode
model.eval()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


PeftModel(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250121, 512)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250121, 512)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    

In [33]:
test_data_ = test_data.sample(10)

outputs = inference_from_csv(model, test_data_, tokenizer, batch_size=8, max_seq_len=512, column_prompt="prompt", use_amp=True)
for item in outputs:
    print("Prompt:", item["prompt"])
    print("Generated Text:", item["generated_text"])
    print("-" * 80)

Prompt: <la> Prima tabula est Baptismus, ubi deponitur vetus homo, et induitur novus; secunda, Poenitentia, qua post lapsum resurgimus, dum vetustas reversa repellitur, et novitas perdita resumitur. Post Baptismum prolapsi per Poenitentiam renovari valent, sed non per Baptismum. Licet homini saepius poenitere, sed non baptizari. Baptismus tantum est sacramentum; sed Poenitentia dicitur et sacramentum, et virtus mentis. Est enim Poenitentia interior, et est exterior. Exterior, sacramentum est; interior, virtus mentis est; et utraque causa salutis est et justificationis. Utrum vero omnis exterior poenitentia sit sacramentum, vel si non omnis, quae hoc nomine censenda sit, consequenter investigabimus. A poenitentia coepit Joannis praedicatio dicentis: #Poenitentiam agite; appropinquabit enim regnum coelorum.@# Quod autem praeco docuit, illud post Veritas praedicavit, exordium sumens sermonis a poenitentia. Poenitentia dicitur a puniendo, qua quis punit illicita quae commisit. Poenitentiae

In [34]:
# Remove every special token from the tokenizer ie every <...>, using regex
pattern = r"<.*?>"
test_data_ = test_data.sample(2000).reset_index(drop=True)
test_data_ = inference_from_csv_adding_column(model, 
                                              test_data_, 
                                              tokenizer, 
                                              batch_size=8, 
                                              max_seq_len=max_new_tokens, 
                                              column_prompt="prompt",
                                              new_column="generated_text", 
                                              use_amp=True)
test_data_ = test_data_.replace(to_replace=pattern, value="", regex=True)
test_data_ = test_data_.replace(to_replace=r"\n", value=" ", regex=True)
test_data_ = test_data_.replace(to_replace=r"\s+", value=" ", regex=True) # remove multiple spaces
test_data_ 

,prompt,answer,prefix,generated_text
0,lotis prius intestinis et pedibus totumque si...,Victory for Thebes is certain. Fear not. Neith...,la.en,But when they shall be satisfied with their fe...
1,Nec parasitorum nobis assentatio in comoediis...,Nec parasitorum nobis assentatio in comoediis ...,la.la,Nec parasitorum nobis assentatio in comoediis ...
2,"Hoc igitur primitus cognito, quod ea ipsa qua...","Hoc igitur primitus cognito, quod ea ipsa quae...",la.la,"Hoc igitur primitus cognito, quod ea ipsa quae..."
3,"Ergo raptorem filiae meae, violatorem foederi...","Seventy-two thousand oxen,",la.en,And when they were a king of the tribe of thei...
4,"Dum patimur, leguntur; dum recognoscimus, pro...","While we suffer, it is all read in the book; t...",la.en,"But when they shall be satisfied, which would ..."
...,...,...,...,...
1995,Datum V Idus Julias per manum Sergii biblioth...,Datum V Idus Julias per manum Sergii bibliothe...,la.la,Datum V Idus Julias per manum Sergii bibliothe...
1996,egressus ad eos Loth post tergum adcludens os...,These are the men who spin out law cases when ...,la.en,"And when they went out of the town, and took o..."
1997,"Homo similiter Deus esse voluit, cui persuasu...","Homo similiter Deus esse voluit, cui persuasum...",la.la,"Homo similiter Deus esse voluit, cui persuasum..."
1998,Adstitit enim mihi quidam candido praeclarus ...,"When this was done, and they had all made a co...",la.en,"But when they would be suitable to us, which s..."


In [ ]:
bleu_score, chrf_score = calculate_bleu_and_chrf(
    test_data_, "en", "la", tokenizer, model,
    max_examples_to_test=500,
    column_prompt="prompt", column_target="answer", column_prefix="prefix",
    max_input_len=max_seq_len, max_output_len=max_new_tokens
)
print(f"BLEU Score: {bleu_score}, CHRF Score: {chrf_score}")

bleu_score, chrf_score = calculate_bleu_and_chrf(
    test_data_, "la", "en", tokenizer, model,
    max_examples_to_test=500,
    column_prompt="prompt", column_target="answer", column_prefix="prefix",
    max_input_len=max_seq_len, max_output_len=max_new_tokens
)
print(f"BLEU Score: {bleu_score}, CHRF Score: {chrf_score}")

## Inference on summaries with mistral 'en' -> 'en'

In [ ]:
from utils.summary_mistral import load_model, mistral_summarize_texts

In [ ]:
# Global parameters for Mistral summarization
MODEL_PATH = "/Data/AxelDlv/Mistral-7B-Instruct-v0.3"
tokenizer_name = "tokenizer.model.v3"
DOWNLOAD_MODEL = False

instruction = (
    "Summarize the following text in a clear and concise manner while preserving its core meaning and key details. "
    "Focus on capturing the main ideas, avoiding unnecessary repetitions, and ensuring coherence. "
    "Keep the summary informative and fluent while maintaining the original context."
)
max_new_tokens = 1000
temperature = 0.3


In [ ]:
# Load the Mistral model and tokenizer
model_mistral, tokenizer_mistral = load_model(MODEL_PATH, tokenizer_name, DOWNLOAD_MODEL)
model_mistral.eval()

In [ ]:
text_to_summarize = "The quick brown fox jumps over the lazy dog."
summary = mistral_summarize_texts(
    text_to_summarize,
    model_mistral,
    tokenizer_mistral,
    max_tokens=max_new_tokens,
    temperature=temperature,
    instruction=instruction
)

print(f"Input: {text_to_summarize}")
print(f"Summary: {summary}")